In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50

dataset_path = os.listdir('Dataset')
degrees = os.listdir('Dataset')


ad = []
for item in degrees:
    if item != 'desktop.ini':
        all_types = os.listdir('Dataset' + '/' + item)
        for type in all_types:
            ad.append((item, str('Dataset' + '/' + item) + '/' + type))
        

ad_df = pd.DataFrame(data=ad, columns = ['degree of disease', 'image'])

ad_count = ad_df['degree of disease'].value_counts()



import os
import cv2
path = 'Dataset'
img_size=224
num_channels=3
img_shape=(224, 224, 3)
images=[]
labels=[]
for degree in degrees:
    if degree != 'desktop.ini':
        data_path = os.path.join(path, str(degree))
        filenames = [f for f in os.listdir(data_path) if f.endswith('.jpg')]
        for filename in filenames:
            img_path = os.path.join(data_path, filename)
            img = cv2.imread(img_path)
        
            if img is not None:
                img = cv2.resize(img, (img_size, img_size))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Ensure correct color channels
                images.append(img)
                labels.append(degree)
            else:
                print(f"Error loading image: {img_path}")

    
    
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y = ad_df['degree of disease'].values

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)

y = y.reshape(-1, 1)
onehotencoder = OneHotEncoder()
Y = onehotencoder.fit_transform(y).toarray()



from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
images, Y = shuffle(images,Y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.6, random_state=123)
Y_np = np.array(Y)
images_shuffled, Y_shuffled = shuffle(images, Y_np, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images_shuffled, Y_shuffled, test_size=0.05, random_state=415)
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)



# Transform both training and test/validation data
train_x_normalized = train_x / 255.0
test_x_normalized = test_x / 255.0
train_y_normalized = train_y
test_y_normalized = test_y

onehotencoder = OneHotEncoder()
train_y_encoded = onehotencoder.fit_transform(train_y.reshape(-1, 1)).toarray()
test_y_encoded = onehotencoder.transform(test_y.reshape(-1, 1)).toarray()


import tensorflow as tf
num_classes = 2
learning_rate = 0.001
training_iters = 10
batch_size = 50



img_size=224
num_channels=3
img_shape=(224, 224, 3)
from tensorflow.keras import layers, models

# Define your model using Keras layers
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=img_shape, pooling='max')
# base_model.trainable = False
model = Sequential([
    base_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(num_classes, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()
n_epochs = 20


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# Train the model with validation split and early stopping
history = model.fit(train_x_normalized, train_y_normalized,
                    epochs=n_epochs, batch_size=batch_size, verbose=1,
                    validation_data=(test_x_normalized, test_y_normalized),
                    shuffle=False, callbacks=[early_stopping])

print("Validation Loss:", history.history['val_loss'])
print("Validation Accuracy:", history.history['val_accuracy'])

model.save('alzheimers.h5')

# Save the model if needed
# model.save('your_model.h5')





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 24120962 (92.01 MB)
Trainable params: 24063746 (91.80 MB)
Non-trainable params: 57216 (223.50 KB)
______

KeyboardInterrupt: 